In [ ]:
## Chap 13 Naive Bayes

#### A Really Dumb Spam Filter

Let `S` be the event "the message is spam" and `V` be the event "the message contains the word *viagra*.". 
`P(S/V) = P(message is spam and contains viagra/ probability that message contains viagra)`

If we have a large collection of messages we know are spam, and a large collection we know are not spam, we can estimate `P(V|S)` and `P(V|¬S)`. If we further assume that any message is equally likely to be spam or not-spam (so that `P(S) = P(¬S) = 0.5`), then:

`P(S|V) = P(V|S) / P(V|S) + P(V|¬S)`

E.g if 50% of spam messages have the word *viagra*, but only 1% of nonspam messages do, then the probability that any given *viagra*-containing email is spam is:


In [92]:
(0.5/(0.5 +0.01)) # 98%

0.9803921568627451

## Implimentation

Building the classifier. First, we will make a function to tokenize messages into distinct words. We'll first convert each message to lowercase; use `re.findall()` to extract "words" consisting of letters, numbers and apostrophes; finally use `set()` to get just the distinct words:

In [135]:
def tokenize(message):
    message = message.lower() # convert to lowercase
    all_words = re.findall("[a-z0-9]+", message) # extract the words
    return set(all_words) # remove duplicates   

Next function will count the words in a labeled training set of messages. We'll have to return a dictionary whose keys are words, and whose values are two-element lists[spam_count, non_spam_count] corresponding to how many times we saw that word in both spam and nonspam messages:

In [136]:
def count_words(training_set):
    """training set consists of pairs (message, is_spam)"""
    counts = defaultdict(lambda:[0, 0])
    for message, is_spam in training_set:
        for word in tokenize(message):
            counts[word][0 if is_spam else 1] += 1 # does dict[][] reference?
    return counts

In [154]:
def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
    """turn the word_counts into a list of triplets
    w,(p(w | spam) and p(w | ~spam))"""
    return[(w,(spam + k) / (total_spams + 2 * k),
           (non_spam + k) / (total_non_spams + 2 * k)) for w, (spam, non_spam) in counts.items()]


The next method uses our `word_probs` classification, determined with the training set, and applies it to a new message to be tested: `message`

In [155]:
def spam_probability(word_probs, message):
    message_words = tokenize(message) # split the message up into each word
    log_prob_if_spam = log_prob_if_not_spam = 0.0
    
    
    # iterate through each word in our vocabulary dict
    for word, prob_if_spam, prob_if_not_spam in word_probs:
        
        # if *word* appears in the message, add the log probability of seeing it
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_spam)
            
        # if *word* doesnt sppear in the message add the log probability of _not_
        # seeing it which is log(1 - probability of seeing it)
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)
        
        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_not_spam = math.exp(log_prob_if_not_spam)
        return prob_if_spam / (prob_if_spam + prob_if_not_spam)        

Put all of this together into a Naive Bayes Classifier:

In [156]:
class NaiveBayesClassifier:
    
    def __init__(self, k=0.5): # k is the psuedo count for smoothing
        self.k = k
        self.words_probs = []
        
    def train(self, training_set):
        
        # count spam and non-spam messages
        num_spams = len([is_spam for message, is_spam in training_set if is_spam])
        num_non_spams = len(training_set) - num_spams
        
        # run training data through our "pipeline"
        word_counts = count_words(training_set)
        self.word_probs = word_probabilities(word_counts, num_spams, num_non_spams, self.k)
        print(list(self.word_probs)[0:10])
        
    def classify(self, message):
        return spam_probability(self.word_probs, message) # when called only need to give it the message to be tested

## Testing Our Model

How do we identify the subject line? Looking through the files, they all seem to start with "Subject" - so we'll look for that.

In [157]:
from collections import Counter, defaultdict
from machine_learning import split_data
import math, random, re, glob

# path for the files
path = r"/home/sophie/projects/DS_fromScratch/*/*"

data = []

# regex for stripping out the leading "Subject:" and any spaces after it
subject_regex = re.compile(r"^Subject:\s+")

# glob.glob returns every filename that matches the wildcarded path
# loops over each file in path
for fn in glob.glob(path):
    #print("fn=%s"%fn)
    is_spam = "ham" not in fn # if "ham" is not in the filename, sets it True

    with open(fn,'r',encoding='ISO-8859-1') as file:
        for line in file:
            if line.startswith("Subject:"):
                subject = subject_regex.sub("", line).strip()
                data.append((subject, is_spam))
    

Now we can split the data into a training data and test data, and then we're ready to build a classifier

In [158]:
random.seed(0)  # to check answers with the example I am following
train_data, test_data = split_data(data, 0.75)

classifier = NaiveBayesClassifier()
classifier.train(train_data)

num_spams = 369
num_non_spams = 2178
counts =  ['kissinger', 'truck', 'orgns']
[('kissinger', 0.0013513513513513514, 0.0006883891693437356), ('truck', 0.0013513513513513514, 0.0011473152822395595), ('orgns', 0.0013513513513513514, 0.0006883891693437356), ('delivers', 0.0013513513513513514, 0.002065167508031207), ('angry', 0.0013513513513513514, 0.0029830197338228544), ('newby', 0.0013513513513513514, 0.0011473152822395595), ('zone', 0.0013513513513513514, 0.0006883891693437356), ('tiling', 0.0013513513513513514, 0.0006883891693437356), ('university', 0.004054054054054054, 0.0002294630564479119), ('interviews', 0.0013513513513513514, 0.0011473152822395595)]


And then we can check how our model does:

In [159]:
# triplets (subject, actual is_spam, predicted spam probability)
# Here we are feeding the classifier object test data.
classified = [(subject, is_spam, classifier.classify(subject)) for subject, is_spam in test_data]

# assume that spam_probability > 0.5 corresponds to spam prediction and count the 
# combinations of (actual is_spam, predicted is_spam)
counts = Counter((is_spam, spam_probability > 0.5) for _,  is_spam, spam_probability in 
                 classified)

In [168]:
print(counts)

Counter({(False, False): 742, (True, False): 134})


Let's also look at the most misclassified:

In [169]:
# sort by spam_probability from smallest to largest
classified.sort(key=lambda row: row[2])

# the highest predicted spam probabilities among the non-spams
# row[1]= True/False is_spam. If False, then not False is True and it is stored in spammiest_hams
spammiest_hams = list(filter(lambda row: not row[1], classified))[-5:]

In [163]:
# the lowest predicted spam probabilities among the actual spams
hammiest_spams = list(filter(lambda row: row[1], classified))[:5]

In [174]:
print(spammiest_hams) # the messages found in ham which had a high % probability of being spam
print("------------")
print(hammiest_spams) # the messages in spam which have the highest probability of not being spam.

[('Core Java Technologies Tech Tips, September 10, 2002 (ArrayList vs. LinkedList, Zero-Length Arrays)', False, 0.49983409024807623), ('F2M - Ihre kostenlose Faxnummer - Newsletter', False, 0.49983409024807623), ("Market Can't Find Its Bottom With Both Hands", False, 0.49983409024807623), ('[Lockergnome Apple Core]  Skip Content', False, 0.49983409024807623), ("Re: Hi! I'm new here.", False, 0.49983409024807623)]
------------
[('New Version 7: Uncover the TRUTH about ANYONE!', True, 0.49983409024807623), ('Low Price Smokes', True, 0.49983409024807623), ('Dont waste your TIME!!! Why?', True, 0.49983409024807623), ('Toners and inkjet cartridges for less....               NOAZ', True, 0.49983409024807623), ('zzzz,All New! Breast Enhancement', True, 0.49983409024807623)]


Similarly we can look at the spammiest *words*:

In [165]:
def p_spam_given_word(word_prob):
    """uses bayes's theorem to compute p(spam | message contains word)"""
    
    # word_prob is one of the triplets produced by word_probabilities
    word, prob_if_spam, prob_if_not_spam = word_prob
    return prob_if_spam / (prob_if_spam + prob_if_not_spam) # calculating bayes theorem here.

# This is sorted by applying bayes theorem to each 
words = sorted(classifier.word_probs, key=p_spam_given_word) # classifier.word_probs = self.word_probs is a class 
                                                             # attribute.
spammiest_words = words[-5:]
hammiest_words = words[:5]

In [173]:
# words most likely to sppear in spam
print(spammiest_words) # this prints out word_probs triplets, sorted by application of bayes theorem to the 2nd and 3rd
                       # elements
print("---------------")

# words most likely to appear in ham
print(hammiest_words)

[('clearance', 0.022972972972972974, 0.0002294630564479119), ('sale', 0.02837837837837838, 0.0002294630564479119), ('systemworks', 0.033783783783783786, 0.0002294630564479119), ('money', 0.03918918918918919, 0.0002294630564479119), ('adv', 0.041891891891891894, 0.0002294630564479119)]
---------------
[('satalk', 0.0013513513513513514, 0.0502524093620927), ('spambayes', 0.0013513513513513514, 0.047957778797613586), ('users', 0.0013513513513513514, 0.03832033042680129), ('was', 0.0013513513513513514, 0.037402478201009635), ('razor', 0.0013513513513513514, 0.032354290959155575)]


How could we get better performance? 1) Get more data to train on. 2) improve the model. Some ways to do this:

- Look at message content, not just the subject line.
- Modify the classifier to accept an optional `min_count` threshold and ignore tokens that don't appear at least that many times
- The tokenizer has no notion of similar words (e.g. "cheap" and "cheapest"). Modify the classifer to take an optional stemmer function that converts words to `equivalence classes` of words. For example, a really simple stemmer function might be:

In [167]:
def drop_final_s(word):
    return re.sub("S$", "", word)

Creating a good stemmer function is hard. People frequently use the [`Porter Stemmer`](http://tartarus.org/martin/PorterStemmer/)